## Setup and Env Sanity check

In [ ]:
# Run the following command to install our version of the environment
!pip install git+https://github.com/joyparikh/ELEN6885_Project

# Run the following to get baselines installed directly from the source (doing a pip install runs into some mujoco-py related error)
#!pip install git+https://github.com/openai/baselines

  Cloning https://github.com/joyparikh/ELEN6885_Project to /tmp/pip-req-build-8z33r_wp
  Running command git clone -q https://github.com/joyparikh/ELEN6885_Project /tmp/pip-req-build-8z33r_wp
  Created wheel for gym-Rubiks-Cube: filename=gym_Rubiks_Cube-0.0.1-py3-none-any.whl size=6985 sha256=034bdaa113666a5f1d11b78d49d94ef0043bc0c7a9e08bac6d5655055010e356
  Stored in directory: /tmp/pip-ephem-wheel-cache-wu2wlpcz/wheels/ab/49/fd/daa07c1c8b5ce52abb609bea2f974b71e552a6ac20545ca66d
Successfully built gym-Rubiks-Cube


In [ ]:
# Import the gym and rubiks cube environment for gym
import gym
import gym_Rubiks_Cube

# Make the environment, replace 2x2 with 4x4. Or remove 2x2 for the normal 3x3 version
env = gym.make("RubiksCube-v1")

# Checkk environment
env.setScramble(1, 1, True)
print(env.reset()) # print status
env.render(mode = 'rgb_array')

for i in _:
  k = env.step(i)

  if k[2]==True:
    print(k[1]) # action 0~11
#env.step(1) # action 0~11
#env.render()

[5 5 5 5 5 5 0 0 0 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 5 5 5 0 0 0 0 0 0 2 2 2 3
 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4]
          +  +  +                   
          +  +  +                   
          +  +  +                   
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
          +  +  +                   
          +  +  +                   
          +  +  +                   



In [ ]:
# import useful stuff
import collections
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

## Helper functions for Actor-Critic Implementation

In [ ]:
# Normal Actor Critic implementation
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common1 = layers.Dense(num_hidden_units, activation="relu")
    self.common2 = layers.Dense(num_hidden_units, activation="relu")
    self.common3 = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x1 = self.common1(inputs)
    x2 = self.common2(x1)
    x = self.common3(x2)
    return self.actor(x), self.critic(x)

# Wrap OpenAI Gym's `env.step` call as an operation in a TensorFlow function.
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward, done, _ = env.step(action)
  return (state.astype(np.float32), 
          np.array(reward, np.int32), 
          np.array(done, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32, tf.int32])
  
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)

    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)

    # Sample next action from the action probability distribution
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])

    # Apply action to the environment to get next state and reward
    state, reward, done = tf_env_step(action)
    state.set_shape(initial_state_shape)

    # Store reward
    rewards = rewards.write(t, reward)

    if tf.cast(done, tf.bool):
      break

  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()

  return action_probs, values, rewards

def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined actor-critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculating loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

## Training

In [ ]:
%%time
# Create the environment - Experiment 1
# Change to v1 and v2 for 2 other environments
env = gym.make("RubiksCube-v0")

# Set seed for experiment reproducibility
seed = 42
env.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()
num_actions = env.action_space.n  # 12
num_hidden_units = 500
model = ActorCritic(num_actions, num_hidden_units)

min_episodes_criterion = 1000
max_episodes = 100000
max_steps_per_episode = 100
 
# Change for different experiments
max_scramble_train = 1

# consecutive trials
reward_threshold = 0.93
running_reward = 0

# Discount factor for future rewards
gamma = 0.98

# Keep last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

with tqdm.trange(max_episodes) as t:
  for i in t:
    env.setScramble(1, max_scramble_train, True)
    env.setStepMax(max_scramble_train*3 + 1)
    initial_state = tf.constant(env.reset(), dtype=tf.float32)
    episode_reward = int(train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode))

    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)

    t.set_description(f'Episode {i}')
    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)

    # Show average episode reward every 10 episodes
    if i % 10 == 0:
      pass # print(f'Episode {i}: average reward: {avg_reward}')

    if running_reward > reward_threshold and i >= min_episodes_criterion:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')


Episode 99999: 100%|██████████| 100000/100000 [35:19<00:00, 47.18it/s, episode_reward=0, running_reward=0.131]


Solved at episode 99999: average reward: 0.13!
CPU times: user 41min 22s, sys: 2min 51s, total: 44min 13s
Wall time: 35min 19s


In [ ]:
import pickle
from google.colab import files

name = 'model_pkl_exp1'
with open(name, 'wb') as obj:
    pickle.dump(model, obj)
files.download('/content/'+name)

INFO:tensorflow:Assets written to: ram://1cb92d3b-e8f0-4984-b456-4c457163313e/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open(name , 'rb') as f:
    lr = pickle.load(f)

## Test

In [ ]:

# Render an episode and save as a GIF file

from IPython import display as ipythondisplay
from PIL import Image
#from pyvirtualdisplay import Display


#display = Display(visible=0, size=(400, 300))
#display.start()


env.setScramble(1, 1, True)
print(env.reset()) # print status
env.render()


state = tf.constant(env.reset(), dtype=tf.float32)
for i in range(1, max_steps + 1):
  state = tf.expand_dims(state, 0)
  action_probs, _ = model(state)
  action = np.argmax(np.squeeze(action_probs))

  state, _, done, _ = env.step(action)
  state = tf.constant(state, dtype=tf.float32)

  # Render screen every 10 steps
  if i % 10 == 0:
    env.render(mode='rgb_array')

  if done:
    break


## test scrambles

In [ ]:
for j in range(1,6):
  count = 0
  for i in range(1000):
    env.setScramble(1, j, True)
    state = tf.constant(env.reset(), dtype=tf.float32)
    for i in range(1, 99 + 1):
      state = tf.expand_dims(state, 0)
      action_probs, _ = lr(state)
      action = np.argmax(np.squeeze(action_probs))

      state, reward, done, _ = env.step(action)
      state = tf.constant(state, dtype=tf.float32)
      
      if done and reward ==1 :
        count = count+reward
        break

  print(count/10,' %')

0.0  %
0.0  %
0.0  %
0.0  %
0.0  %


In [ ]:
%%time
# Create the environment - Experiment 1
# Change to v1 and v2 for 2 other environments
env = gym.make("ERubiksCube-v2")

# Set seed for experiment reproducibility
seed = 42
env.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()
num_actions = env.action_space.n  # 12
num_hidden_units = 100
model = ActorCritic(num_actions, num_hidden_units)

min_episodes_criterion = 1000
max_episodes = 100000
max_steps_per_episode = 100
 
# Change for different experiments
max_scramble_train = 5

# consecutive trials
reward_threshold = 1
running_reward = 0

# Discount factor for future rewards
gamma = 0.9

# Keep last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

with tqdm.trange(max_episodes) as t:
  for i in t:
    env.setScramble(1, max_scramble_train, True)
    initial_state = tf.constant(env.reset(), dtype=tf.float32)
    episode_reward = int(train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode))

    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)

    t.set_description(f'Episode {i}')
    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)

    # Show average episode reward every 10 episodes
    if i % 10 == 0:
      pass # print(f'Episode {i}: average reward: {avg_reward}')

    if running_reward > reward_threshold and i >= min_episodes_criterion:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')


Episode 99999: 100%|██████████| 100000/100000 [2:07:28<00:00, 13.07it/s, episode_reward=-100, running_reward=-89.8]


Solved at episode 99999: average reward: -89.85!
CPU times: user 2h 50min 28s, sys: 6min 57s, total: 2h 57min 25s
Wall time: 2h 7min 28s


In [ ]:
import pickle
from google.colab import files

name = 'model_pkl_exp2'
with open(name, 'wb') as obj:
    pickle.dump(model, obj)
files.download('/content/'+name)

INFO:tensorflow:Assets written to: ram://a80dea40-426f-4274-8918-e72b7c512f59/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open(name , 'rb') as f:
    lr = pickle.load(f)